# 📊 **Valuation of Adobe Inc. Using Python, Plotly & DCF Analysi**s

This notebook estimates the intrinsic value of Adobe Inc. using:
- Free cash flow (FCF) from financial statements
- Cost of equity via CAPM
- WACC and Terminal Value
- Plotly visualizations for snapshot & waterfall analysis

---


In [1]:
import yfinance as yf           # For financial data
import pandas as pd             # For data manipulation
import numpy as np              # For calculations (NaN, etc.)
import plotly.graph_objects as go     # For Plotly charts
from plotly.subplots import make_subplots   # For subplot visualizations
from datetime import datetime, timedelta     # For time range in charts

In [2]:
from adobe_valuation import run_valuation

val = run_valuation()

print(f"Intrinsic Value per Share: ${val['intrinsic_value_per_share']:.2f}")


Intrinsic Value per Share: $150.68


In [3]:
intrinsic_value_per_share = val["intrinsic_value_per_share"]
current_price = val["current_price"]



# 📊 **Price vs Intrinsic Value Chart**

In [4]:
import plotly.graph_objects as go

# Compute over/undervaluation
intrinsic = val["intrinsic_value_per_share"]
market = val["current_price"]
delta_val = intrinsic - market
delta_pct = (delta_val / market) * 100

# Define valuation direction
verdict = "Undervalued" if delta_val > 0 else "Overvalued"
delta_color = "green" if delta_val > 0 else "red"

fig = go.Figure(data=[go.Table(
    header=dict(
        values=["<b>Metric</b>", "<b>Value</b>"],
        fill_color="#2a3f5f",
        font=dict(color='white', size=13),
        align="center"
    ),
    cells=dict(
        values=[
            ["Intrinsic Value/Share", "Current Market Price", "Valuation Difference", "Valuation Verdict"],
            [f"${intrinsic:.2f}", f"${market:.2f}", f"{delta_pct:.2f}%", verdict]
        ],
        fill_color=[["white", "white", "white", delta_color]],
        align="center",
        height=40,
        font=dict(size=14)
    )
)])

fig.update_layout(title="📊 Adobe Valuation Summary", width=600, height=300)
fig.show()
fig.write_html("valuation_summary.html")


### **Key Takeaway:**
**Intrinsic Value Estimate:** ~$150/share

**Current Market Price:** ~$412/share

**Valuation Verdict:** The model suggests Adobe is overvalued by ~63%.

**Implication:** Investors may be pricing in more growth or lower risk than assumed in the DCF model.

# 📉**DCF Waterfall Chart**

In [5]:
waterfall_data = [
    {"label": "Free Cash Flow", "value": val["fcf"]},
    {"label": "Terminal Value", "value": val["terminal_value"]},
    {"label": "PV Adjustment", "value": -val["terminal_value"] + val["present_value_tv"]},
    {"label": "Add: Cash", "value": val["cash"]},
    {"label": "Less: Debt", "value": -val["debt"]},
    {"label": "Equity Value", "value": val["intrinsic_equity_value"]},  # Final total
]

fig2 = go.Figure(go.Waterfall(
    name="DCF",
    orientation="v",
    measure=["relative", "relative", "relative", "relative", "relative", "total"],
    x=[d["label"] for d in waterfall_data],
    y=[d["value"] for d in waterfall_data],
    connector={"line": {"color": "gray"}}
))

fig2.update_layout(
    title="📉 DCF Valuation Breakdown (Waterfall)",
    showlegend=False,
    template="plotly_white",
    height=400,
    width=800
)

fig2.show()
fig2.write_html("dcf_waterfall.html")


### **Key Takeaway:**
**Largest Component:** Terminal value makes up the majority of Adobe’s firm value.

**Cash & Debt:** Cash adds to equity value, while debt reduces it.

**Overall Takeaway:** Equity value is highly sensitive to the terminal value and WACC assumptions.

**Recommendation:** Carefully examine the assumptions driving terminal value.

# 📈 **Historical Financials**

In [11]:
def plot_historical_financials(ticker_symbol="ADBE"):
    ticker = yf.Ticker(ticker_symbol)
    revenue = ticker.financials.loc["Total Revenue"]
    net_income = ticker.financials.loc["Net Income"]
    fcf = ticker.cashflow.loc["Free Cash Flow"]

    fig3 = make_subplots(rows=1, cols=3, subplot_titles=("Total Revenue", "Net Income", "Free Cash Flow"))
    fig3.add_trace(go.Bar(x=revenue.index, y=revenue.values, name="Revenue"), row=1, col=1)
    fig3.add_trace(go.Bar(x=net_income.index, y=net_income.values, name="Net Income"), row=1, col=2)
    fig3.add_trace(go.Bar(x=fcf.index, y=fcf.values, name="Free Cash Flow"), row=1, col=3)
    fig3.update_layout(title_text="📈 Historical Financials", height=400)
    return fig3

# Create and save the plot
plot_historical_financials.show()
plot_historical_financials().write_html("historical_financials.html")

AttributeError: 'function' object has no attribute 'show'

### **Key Takeaway:**
**Revenue:** Steady increase year-over-year, indicating healthy top-line growth.

**Net Income:** Consistent profitability with mild year-to-year fluctuations.

**Free Cash Flow (FCF):** Adobe maintains strong FCF, supporting valuation confidence.

**Insight:** Historical performance provides a solid foundation for projecting future cash flows.

# 🤝 **Peer Comparables**

In [ ]:
def plot_peer_comparables(tickers=["ADBE", "MSFT", "ORCL", "CRM"]):
    data = []
    for symbol in tickers:
        t = yf.Ticker(symbol)
        info = t.info
        data.append({
            "Company": symbol,
            "P/E": info.get("trailingPE", np.nan),
            "P/B": info.get("priceToBook", np.nan),
            "ROE": info.get("returnOnEquity", np.nan),
            "Profit Margin": info.get("profitMargins", np.nan)
        })
    df = pd.DataFrame(data)
    fig4 = make_subplots(rows=2, cols=2, subplot_titles=["P/E Ratio", "P/B Ratio", "Return on Equity", "Profit Margin"])
    fig4.add_trace(go.Bar(x=df["Company"], y=df["P/E"], name="P/E"), row=1, col=1)
    fig4.add_trace(go.Bar(x=df["Company"], y=df["P/B"], name="P/B"), row=1, col=2)
    fig4.add_trace(go.Bar(x=df["Company"], y=df["ROE"], name="ROE"), row=2, col=1)
    fig4.add_trace(go.Bar(x=df["Company"], y=df["Profit Margin"], name="Profit Margin"), row=2, col=2)
    fig4.update_layout(title_text="📉 Peer Comparables", height=600)
    fig4.show()

plot_peer_comparables()
plot_peer_comparables.write_html("peer_comparables.html")

### **Key Takeaway:**
**P/E & P/B Ratios:** Adobe has a higher valuation multiple than some peers (e.g., MSFT, ORCL).

**ROE & Profit Margin:** Adobe demonstrates solid profitability, especially on margins.

**Positioning:** Adobe is strong on fundamentals but priced at a premium.

**Investor View:** Premium pricing may be justified by brand strength or growth expectations.


# 🔍**Risk & Growth Analysis**

In [ ]:
def plot_risk_and_growth(ticker_symbol="ADBE"):
    ticker = yf.Ticker(ticker_symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5*365)
    hist = ticker.history(start=start_date, end=end_date)
    returns = hist["Close"].pct_change()
    volatility = returns.rolling(window=30).std() * np.sqrt(252)

    revenue = ticker.financials.loc["Total Revenue"]
    revenue_growth = revenue.pct_change() * 100

    fig5 = make_subplots(rows=1, cols=2, subplot_titles=["Rolling Volatility", "Revenue Growth Rate"])
    fig5.add_trace(go.Scatter(x=volatility.index, y=volatility, name="Volatility"), row=1, col=1)
    fig5.add_trace(go.Bar(x=revenue_growth.index, y=revenue_growth, name="Revenue Growth"), row=1, col=2)
    fig5.update_layout(title_text="🔍 Risk & Growth Analysis", height=400)
    fig5.show()

plot_risk_and_growth()
plot_risk_and_growth.write_html("risk_growth_analysis.html")

### **Key Takeaway:**
**Volatility:** Rolling 30-day volatility shows moderate price fluctuations.

**Revenue Growth Rate:** Recent slowdown in growth could be a concern.

**Macro Implication:** Revenue sensitivity and volatility affect investor risk perception.

**Actionable Insight:** Monitor future earnings and revenue acceleration to validate high valuation.

## 🧾 **Final Summary**

This valuation analysis of Adobe Inc. used Discounted Cash Flow (DCF) methodology alongside peer comparables and historical financials. The intrinsic value was estimated based on projected free cash flows, terminal growth, and WACC assumptions.

- **Valuation Verdict:** The model estimates Adobe's fair value at ~$150/share vs a market price of ~$412, indicating the stock may be **significantly overvalued** under current assumptions.
- **Drivers:** Terminal value accounts for the bulk of total firm value.
- **Financials:** Adobe has shown consistent revenue and FCF growth.
- **Comparables:** Adobe has strong profit margins but may lag peers in return metrics.
- **Risks:** Market volatility is moderate, but growth rates are tapering.

📌 This analysis provides a base valuation model. Investors should consider macroeconomic factors, competitive landscape, and management execution before making any investment decision.


In [ ]:
# Export key valuation metrics to Excel
df_export = pd.DataFrame({
    "Metric": list(val.keys()),
    "Value": [f"${v:,.2f}" if isinstance(v, (int, float)) else v for v in val.values()]
})

df_export.to_excel("adobe_valuation_output.xlsx", index=False)
print("✅ Exported summary to 'adobe_valuation_output.xlsx'")